In [2]:
import numpy as np

In [3]:
import os

current_directory = os.getcwd()
print(current_directory)


c:\Users\ks885\Documents\aa_research\Data\Energy\ferc_generator_parameters


In [4]:
import pandas as pd

# Read the Excel file
summer_raw_df = pd.read_excel('20120724-4012_Generator_Data_Summer.xlsx',header=None)

# Display the data
print(summer_raw_df.head())


             0                       1           2                3   \
0           NaN           EIA Data Sets         NaN              NaN   
1  Generic Name  Energy_Source_1 (Fuel)  PRIMEMOVER  NAMEPLATE (MWs)   
2          GEN1                     SUB          ST              660   
3          GEN2                     SUB          ST              660   
4          GEN3                     SUB          ST              660   

                          4                                5   \
0                        NaN  Class averages from PJM website   
1  SEASONAL_CAPABILITY (MWs)                            EFORD   
2                        518                            5.731   
3                        518                            5.731   
4                        518                            5.731   

                                 6                   7   \
0  Statistical Analysis of EPA Data                 NaN   
1                  RAMP UP (MW/min)  RAMP DOWN (MW/min)   

In [5]:
# Get the sheet names
excel_file = pd.ExcelFile('20120724-4012_Generator_Data_Summer.xlsx')
sheet_names = excel_file.sheet_names

# Create a dictionary to store the separated dataframes
separated_dataframes = {}

# Iterate over each sheet and store the data in the dictionary
for sheet_name in sheet_names:
    separated_dataframes[sheet_name] = excel_file.parse(sheet_name)

# Access the dataframes for each sheet
# for sheet_name, df in separated_dataframes.items():
#     print(f"Sheet Name: {sheet_name}")
#     print(df)
#     print()


In [6]:
sheet_names = excel_file.sheet_names
print(sheet_names)


['Generator Characteristics', 'Generator Offer Curve', 'Generator HeatRates', 'Prime Mover Codes', 'Fuel Codes', 'Generator Status']


In [7]:
### Manual Cleaning Processes
# Process for Cleaning Generator Characteristics
gen_characteristics = separated_dataframes['Generator Characteristics'].copy()
gen_characteristics.columns = separated_dataframes['Generator Characteristics'].iloc[0]
gen_characteristics.drop(0, inplace=True)
gen_characteristics.reset_index(drop=True, inplace=True)
# gen_characteristics.drop_duplicates(gen_characteristics.columns[duplicate_index], axis=1)
gen_characteristics= gen_characteristics.loc[:,~gen_characteristics.columns.duplicated()].copy()

In [8]:
print(gen_characteristics.columns)

Index([             'Generic Name',    'Energy_Source_1 (Fuel)',
                      'PRIMEMOVER',           'NAMEPLATE (MWs)',
       'SEASONAL_CAPABILITY (MWs)',                     'EFORD',
                'RAMP UP (MW/min)',        'RAMP DOWN (MW/min)',
           'Economic Minimum (MW)',        'MIN_DOWN_TIME (hr)',
               'MIN_RUN_TIME (hr)',        'CO2 rate (lbs/MWh)',
              'NOx rate (lbs/MWh)',        'SO2 rate (lbs/MWh)',
                               nan],
      dtype='object', name=0)


In [9]:
# the important paramaters are:
key_params = ['RAMP UP (MW/min)', 'RAMP DOWN (MW/min)', 'MIN_RUN_TIME (hr)', 'MIN_DOWN_TIME (hr)']

In [10]:
# process for Cleaning Generator Offer Curve Data
# first separate the price data from the Cost data

mw_offer_curve = separated_dataframes['Generator Offer Curve'].iloc[:,0:11]
# 
mw_offer_curve.columns = mw_offer_curve.iloc[0]
mw_offer_curve.drop(0, inplace=True)
mw_offer_curve.reset_index(drop=True, inplace=True)
mw_offer_curve

,Generic Name,MW1,MW2,MW3,MW4,MW5,MW6,MW7,MW8,MW9,MW10
0,NaN,1,2,3,4,5,6,7,8,9,10
1,GEN1,1,39,38,440,0,0,0,0,0,0
2,GEN2,1,89,60,368,0,0,0,0,0,0
3,GEN3,1,89,60,368,0,0,0,0,0,0
4,GEN4,13,287,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1007,GEN1007,198,0,0,0,0,0,0,0,0,0
1008,GEN1008,10.8,0,0,0,0,0,0,0,0,0
1009,GEN1009,10.8,0,0,0,0,0,0,0,0,0
1010,GEN1010,29.4,0,0,0,0,0,0,0,0,0


In [11]:
price_offer_curve = separated_dataframes['Generator Offer Curve'].iloc[:,12:23]
price_offer_curve.columns = price_offer_curve.iloc[0]
price_offer_curve.drop(0, inplace=True)
price_offer_curve.reset_index(drop=True, inplace=True)
price_offer_curve

,NaN,Price1,Price2,Price3,Price4,Price5,Price6,Price7,Price8,Price9,Price10
0,NaN,1,2,3,4,5,6,7,8,9,10
1,GEN1,16.09,26.64,36.92,151.1,0,0,0,0,0,0
2,GEN2,21.23,25.11,27.72,42.97,0,0,0,0,0,0
3,GEN3,21.23,25.11,27.72,42.97,0,0,0,0,0,0
4,GEN4,250.78,386.69,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1007,GEN1007,0,0,0,0,0,0,0,0,0,0
1008,GEN1008,0,0,0,0,0,0,0,0,0,0
1009,GEN1009,0,0,0,0,0,0,0,0,0,0
1010,GEN1010,0,0,0,0,0,0,0,0,0,0


In [12]:
gen_costs = separated_dataframes['Generator Offer Curve'].iloc[:,24:28]
gen_costs.columns = gen_costs.iloc[1]
gen_costs.drop([0,1], inplace=True)
gen_costs.reset_index(drop=True, inplace=True)
gen_costs.rename(columns={'1': 'index'}, inplace=True)
gen_costs.rename(columns={np.nan: 'Generic Name'}, inplace=True)


In [13]:
gen_costs

1,Generic Name,No Load Cost ($),Cold Start Cost ($),Hot Start Cost ($)
0,GEN1,2200.007004,2700,2700
1,GEN2,2200.00701,2700,2700
2,GEN3,3500.007016,11550,11550
3,GEN4,381.477022,1607.33,1607.33
4,GEN5,0.007028,887.52,887.52
...,...,...,...,...
1006,GEN1007,0.014055,0,0
1007,GEN1008,0.014064,0,0
1008,GEN1009,0.014073,0,0
1009,GEN1010,0.014082,0,0


In [14]:
gen_heat_rates_df = separated_dataframes['Generator HeatRates']
gen_heat_rates_df.rename(columns={'Generic Generator Name': 'Generic Name'}, inplace=True)
gen_heat_rates_df

,Generic Name,Nominal Heat Rate,MW_OUTPUT_1,MMBTU_PER_MWH1,MW_OUTPUT_2,MMBTU_PER_MWH2,MW_OUTPUT_3,MMBTU_PER_MWH3,MW_OUTPUT_4,MMBTU_PER_MWH
0,GEN1,11273.5443,2749231.00,11949.956958,824769.3,10146.18987,1649538.6,10709.867085,274923.1,11273.5443
1,GEN2,11273.5443,2749231.00,11949.956958,824769.3,10146.18987,1649538.6,10709.867085,274923.1,11273.5443
2,GEN4,23185.3494,317.94,23185.349400,NaN,NaN,NaN,NaN,NaN,23185.3494
3,GEN5,11997.6876,1762130.00,11997.687600,NaN,NaN,NaN,NaN,NaN,11997.6876
4,GEN6,11997.6876,881065.00,12717.548856,264319.5,10797.91884,528639.0,11397.803220,88106.5,11997.6876
...,...,...,...,...,...,...,...,...,...,...
945,GEN1007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
946,GEN1008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
947,GEN1009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
948,GEN1010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
prime_mover_codes_df = separated_dataframes['Prime Mover Codes']
prime_mover_codes_df

row = prime_mover_codes_df[prime_mover_codes_df['Prime Mover Codes'] == 'ST']
description = row['Description'].values[0]
description

def print_prime_mover_description(string, prime_mover_codes):
    description = prime_mover_codes.loc[prime_mover_codes['Prime Mover Codes'] == string, 'Description'].values
    if len(description) > 0:
        print(description[0])
    else:
        print("No description found for the given string.")



In [21]:
print_prime_mover_description('IC', prime_mover_codes_df)

Internal Combustion (diesel, piston) Engine


In [17]:
fuel_codes_df = separated_dataframes['Fuel Codes']
fuel_codes_df

def print_fuel_codes_description(string, fuel_codes):
    description = fuel_codes.loc[fuel_codes['Energy Source Code'] == string, 'Description'].values
    if len(description) > 0:
        print(description[0])
    else:
        print("No description found for the given string.")

In [18]:
def get_fuel_mover(data, fuel, mover):
    return data[(data['Energy_Source_1 (Fuel)'] == fuel) & (data['PRIMEMOVER'] == mover)]

In [19]:
energy_source_codes = gen_characteristics['Energy_Source_1 (Fuel)'].unique()
prime_mover_codes = gen_characteristics['PRIMEMOVER'].unique()

for energy in energy_source_codes:
    # print(energy)
    for prime_mover in prime_mover_codes:
        # print(get_fuel_mover(gen_characteristics, energy, prime_mover).shape)
        if get_fuel_mover(gen_characteristics, energy, prime_mover).shape[0] > 0:
            print(f'Energy Source: {energy}, Prime Mover: {prime_mover},','num gen =', get_fuel_mover(gen_characteristics, energy, prime_mover).shape[0])

Energy Source: SUB, Prime Mover: ST, num gen = 26
Energy Source: DFO, Prime Mover: ST, num gen = 4
Energy Source: DFO, Prime Mover: GT, num gen = 164
Energy Source: DFO, Prime Mover: IC, num gen = 40
Energy Source: DFO, Prime Mover: CT, num gen = 1
Energy Source: BIT, Prime Mover: ST, num gen = 163
Energy Source: RFO, Prime Mover: ST, num gen = 21
Energy Source: NUC, Prime Mover: ST, num gen = 41
Energy Source: KER, Prime Mover: GT, num gen = 16
Energy Source: WAT, Prime Mover: HY, num gen = 83
Energy Source: WAT, Prime Mover: PS, num gen = 23
Energy Source: NG, Prime Mover: ST, num gen = 15
Energy Source: NG, Prime Mover: GT, num gen = 237
Energy Source: NG, Prime Mover: IC, num gen = 2
Energy Source: NG, Prime Mover: CT, num gen = 58
Energy Source: NG, Prime Mover: CA, num gen = 33
Energy Source: NG, Prime Mover: CC, num gen = 14
Energy Source: SC, Prime Mover: ST, num gen = 21
Energy Source: WC, Prime Mover: ST, num gen = 14
Energy Source: JF, Prime Mover: GT, num gen = 1
Energy Sou

In [20]:
print_prime_mover_description('IC', prime_mover_codes_df)

Internal Combustion (diesel, piston) Engine
